In [12]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns 
sns.set()

In [40]:
#Find common search parameters excluding year and census location at:
#https://wonder.cdc.gov/controller/saved/D76/D82F692

dfne1999 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/1999_Cause_of_death_Northeast.csv')
# dfne2008 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2008_Cause_of_death_Northeast.csv')
# dfne2018 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2018_Cause_of_death_Northeast.csv')
# dfmw1999 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/1999_Cause_of_death_Midwest.csv')
# dfmw2008 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2008_Cause_of_death_midwest.csv')
# dfmw2018 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2018 Cause_of_death_Midwest.csv')
# dfs1999 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/1999_Cause_of_death_South.csv')
# dfs2008 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2008_Cause_of_death_South.csv')
# dfs2018 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2018_Cause_of_death_South.csv')
# dfw1999 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/1999_Cause_of_death_West.csv')
# dfw2008 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2008_Cause_of_death_West.csv')
# dfw2018 = pd.read_csv('/Users/robholmstrom/Gitrepos/Sams_github/Capstone_1/COD_csv_files/2018_Cause_of_death_West.csv')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf7 in position 9: invalid start byte

Centers for Disease Control and Prevention, National Center for Health Statistics. Underlying Cause of Death 1999-2018 on CDC WONDER Online Database, released in 2020. Data are from the Multiple Cause of Death Files, 1999-2018, as compiled from data provided by the 57 vital statistics jurisdictions through the Vital Statistics Cooperative Program. Accessed at http://wonder.cdc.gov/ucd-icd10.html on May 1, 2020 5:00:25 PM


In [37]:

print(dfw2018.shape)
dfw2018.head(10)


(97, 12)


,Notes,Race,Race Code,Cause of death,Cause of death Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,65-74 years,65-74,Female,F,12.0,Not Applicable,Not Applicable
1,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,65-74 years,65-74,Male,M,19.0,Not Applicable,Not Applicable
2,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,75-84 years,75-84,Female,F,18.0,Not Applicable,Not Applicable
3,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,75-84 years,75-84,Male,M,15.0,Not Applicable,Not Applicable
4,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,85+ years,85+,Female,F,15.0,Not Applicable,Not Applicable
5,NaN,White,2106-3,"Septicaemia, unspecified",A41.9,85+ years,85+,Male,M,14.0,Not Applicable,Not Applicable
6,NaN,White,2106-3,"Pancreas, unspecified - Malignant neoplasms",C25.9,65-74 years,65-74,Male,M,12.0,Not Applicable,Not Applicable
7,NaN,White,2106-3,"Bronchus or lung, unspecified - Malignant neop...",C34.9,55-64 years,55-64,Female,F,13.0,Not Applicable,Not Applicable
8,NaN,White,2106-3,"Bronchus or lung, unspecified - Malignant neop...",C34.9,55-64 years,55-64,Male,M,25.0,Not Applicable,Not Applicable
9,NaN,White,2106-3,"Bronchus or lung, unspecified - Malignant neop...",C34.9,65-74 years,65-74,Female,F,10.0,Not Applicable,Not Applicable
